In [1]:
#pip install hpsklearn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
#pip install hyperopt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
from importlib.machinery import SourceFileLoader
# For Hyperparameter optimization
from hpsklearn import HyperoptEstimator
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

# Loading data
process_test_data = SourceFileLoader('process_test_data', 'D:/UTS_MDSI/ADSI/Part A/src/data/process_test_data.py').load_module()
visualize = SourceFileLoader('visualize', 'D:/UTS_MDSI/ADSI/Part A/src/visualization/visualize.py').load_module()

# optimization to calculate stats and parameter optimization
#use_kaggle_data = False
run_feature_selection = False  # Select the best set of given number of features 
run_parameter_optimization = False

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


Split train datasets for training (80%), testing (10%) and validation (10%) and normalize features using MinMaxScaler.

In [6]:
if use_kaggle_data:
    X_train, y_train, X_test = process_test_data.load_kaggle_train_and_test_data('D:/UTS_MDSI/ADSI/Part A/raw/2022_train.csv', 'D:/UTS_MDSI/ADSI/Part A/raw/2022_test.csv')
else:    
    X_train, y_train, X_test, y_test, X_valid, y_valid = \
    process_test_data.split_and_normalize('D:/UTS_MDSI/ADSI/Part A/raw/2022_train.csv', 'D:/UTS_MDSI/ADSI/Part A/processed/week3')

Original train shape: (8000, 21)
Concat shape: (8000, 20)
Files written to: D:/UTS_MDSI/ADSI/Part A/processed/week3
X_train shape: (6400, 19)
y_train shape: (6400, 1)
X_test shape: (800, 20)
y_test shape: (800, 1)
X_valid shape: (800, 20)
y_valid shape: (800, 1)


Check details of the data if required

In [ ]:
 # X_test.describe()

Selecting features using sequential feature selection if required

In [7]:
if run_feature_selection==True:
    num_of_features_to_select = 13
    features = process_test_data.sequential_feature_selection('D:/UTS_MDSI/ADSI/Part A/raw/train.csv', num_of_features_to_select) #['GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3P Made', '3PA', 'OREB', 'DREB', 'REB', 'BLK', 'TOV']
    X_train = X_train[features]
    # Appending Id column since it should be kept
    features.append('Id')
    X_test = X_test[features]

Running parameter optimization

In [8]:
# Defining pre-identified best parameters if parameter optimization is not going to run
hyp_params = {
'colsample_bytree': 0.65,
'grow_policy': 'lossguide',
'learning_rate': 0.05,
'max_delta_step': 7,
'max_depth': 2, 
'min_child_weight': 5.0,
'min_split_loss': 11,
'subsample': 0.2,
'booster': 'gbtree'     
}

def hyperparameter_tuning(space):
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import roc_auc_score
    
    xgboost = xgb.XGBClassifier(**space, eval_metric="auc", use_label_encoder=False) 
    acc = cross_val_score(xgboost, X_train, y_train, cv=10, scoring='roc_auc').mean()
    return{'loss': 1-acc, 'status': STATUS_OK }

if run_parameter_optimization==True:
    booster = ['gbtree', 'gblinear', 'dart']
    grow_policy = ['depthwise', 'lossguide']
    
    space = {
    "max_depth": hp.choice('max_depth', range(5, 20, 1)),
    "learning_rate": hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    "min_child_weight": hp.quniform('min_child_weight', 1, 10, 1),
    "subsample": hp.quniform('subsample', 0.1, 1, 0.05),
    "colsample_bytree": hp.quniform('colsample_bytree', 0.1, 1.0, 0.05),
    "min_split_loss": hp.choice('min_split_loss', range(0, 20, 1)),
    "max_delta_step": hp.choice('max_delta_step', range(0, 10, 1)),
    "grow_policy": hp.choice("grow_policy", grow_policy),     
    "booster": booster[0]     
    }
    
    # Initialize trials object
    trials = Trials()
    
    best = fmin(fn=hyperparameter_tuning, space = space, algo=tpe.suggest, max_evals=100, trials=trials)
    
    hyp_params['max_depth'] = best['max_depth']
    hyp_params['learning_rate'] = best['learning_rate']
    hyp_params['min_child_weight'] = best['min_child_weight']
    hyp_params['subsample'] = best['subsample']
    hyp_params['colsample_bytree'] = best['colsample_bytree'] 
    hyp_params['min_split_loss'] = best['min_split_loss']
    hyp_params['max_delta_step'] = best['max_delta_step']
    hyp_params['grow_policy'] = grow_policy[best['grow_policy']]
    hyp_params['booster'] = booster[0]
    
    print("Best: {}".format(best))

Training the xgboost

In [9]:
xgboost = xgb.XGBClassifier(**hyp_params, use_label_encoder=False)

# Converting column y values to 1d array
xgboost.fit(X_train, y_train, eval_metric='auc')

C:\Users\Yatindra\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Yatindra\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.65,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=11, gpu_id=-1,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=7,
              max_depth=2, max_leaves=0, min_child_weight=5.0,
              min_split_loss=11, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto', ...)

Predicting using trained random forest 

In [10]:
# Selecting columns to train
test_X = X_test.loc[:, 'GP':'TOV']
# Selecting Ids for CSV
test_X_Ids = X_test.loc[:,'Id']

if use_kaggle_data==True:
    # Predicting probabilities and selecting probability of class 1.
    pred = xgboost.predict_proba(test_X)[:,1]  
else:
    # Predicting classes (1 or 0) for calculating accuracy
    pred = xgboost.predict(test_X) 
    # Probabilities for calculating ROC
    rf_probs = xgboost.predict_proba(test_X)[:,1]

# Data frame with ID for csv writing. In Kaggle mode pred will contains probabilities and else contains classes
result = pd.DataFrame(data = {'Id': test_X_Ids, 'TARGET_5Yrs': pred}) 
# Extracting values for calculating stats
result_values = result[['TARGET_5Yrs']] 

Saving the trainned model and writing result to a CSV file

In [11]:
joblib.dump(xgboost, "D:/UTS_MDSI/ADSI/Part A/models/Yatin_xgboost.joblib", compress=3)

['D:/UTS_MDSI/ADSI/Part A/models/Yatin_xgboost.joblib']

Show stats related to performance of the model if not using Kaggle dataset

In [13]:
#output=data_test[['Id','TARGET_5Yrs']]
result.to_csv('D:/UTS_MDSI/ADSI/Part A/submission/submission_yatinXGB_week3.csv',index=False)

In [12]:
if use_kaggle_data==False:
    visualize.show_random_forest_stats(xgboost, test_X, y_test, rf_probs)
else:
    result.to_csv("D:/UTS_MDSI/ADSI/Part A/submission/submission_yatinXGB_week3.csv", index = False)
   

Average absolute error: 16.625%
ROC: 0.70114
